In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials

from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.metrics import r2_score
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

!pip install holidays_co
from holidays_co import is_holiday_date

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Permisos para leer desde drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#Link del csv con nuestros datos
cars = 'https://docs.google.com/spreadsheets/d/1JgK1Us7vudsKMsR5qpF3z2737ZQOdwgrd8Jdw79mAGs/edit'
 
# to get the id part of the file
id = cars.split("/")[-2]
Miunidad = drive.Miunidad({'id':id})
Miunidad.GetContentFile('registros_autos_entrenamiento.xlsx') 
df = pd.read_excel('registros_autos_entrenamiento.xlsx')

AttributeError: ignored

In [ ]:
df

 **Ingeniería de características**

Como se puede apreciar, los datos leídos están compuestos por dos columnas: Fecha y Unidades.

Fecha: fecha en la que se registraron cierta cantidad de autos en el RUNT.
Unidades: cantidad de vehículos registrados en el RUNT.
Estas fechas guardan información de forma implícita y, por medio de ingeniería de características, esa información será extraída.

Se definen las funciones que permitirán extraer la información de las fechas.

Get_day: devuelve el día del mes.
Get_weekday: devuelve el día de la semana (Lunes, Martes, Miércoles, Jueves, Viernes, Sábado o Domingo).
Get_year: devuelve el año.
Get_month: devuelve el mes.
Get_dayofyear: devuelve el día en el año.
Get_weekofyear: devuelve la semana del año.
Is_weekend: devuelve si es fin de semana o no.
Is_startofmonth: devuelve si es el inicio del mes.
Is_endofmonth: devuelve si es fin de mes.
Is_startofyear: devuelve si es el inicio del año.
Is_endofyear: devuelve si es fin de año.
Estas funciones se aplican en una función mayor llamada "database", la cual construye la base de datos con base en la fecha para utilizarla en los modelos de predicción.

Esta tabla define el valor para cada día de la semana.

image.png

In [ ]:
# function to get all data from time stamp
 
# get date
def get_day(dt):
    return dt.day
 
# get week day
def get_weekday(dt):
    return dt.weekday()
 
# get year
def get_year(dt):
    return dt.year
 
# get month
def get_month(dt):
    return dt.month
 
# get year day
def get_dayofyear(dt):
    return dt.dayofyear
 
# get year week
def get_weekofyear(dt):
    return dt.weekofyear

# verify if its weekend
def is_weekend(day):
  return np.where(day in ([5,6]), 1,0)

# verify if it's start of month
def is_startofmonth(dt):
  return 1 if dt.is_month_start else 0

# verify if it's end of month
def is_endofmonth(dt):
  return 1 if dt.is_month_end else 0

# verify if it's start of year
def is_startofyear(dt):
  return 1 if dt.is_year_start else 0

# verify if it's end of year
def is_endofyear(dt):
  return 1 if dt.is_year_end else 0

def database(df):
  df['Fecha'] = df['Fecha'].map(pd.to_datetime)
  df['Dia'] = df['Fecha'].map(get_day)
  df['Dia_Semana'] = df['Fecha'].map(get_weekday)
  df['Fin_Semana'] = df['Dia_Semana'].map(is_weekend)
  df['Mes'] = df['Fecha'].map(get_month)
  df['Inicio_Mes'] = df['Fecha'].map(is_startofmonth)
  df['Fin_Mes'] = df['Fecha'].map(is_endofmonth)
  df['Anio'] = df['Fecha'].map(get_year)
  df['Inicio_Anio'] = df['Fecha'].map(is_startofyear)
  df['Fin_Anio'] = df['Fecha'].map(is_endofyear)
  df['Dia_Anio'] = df['Fecha'].map(get_dayofyear)
  df['Semana_Anio'] = df['Fecha'].map(get_weekofyear)
  df['Festivo'] = df.Fecha.apply(lambda fecha: 1 if is_holiday_date(fecha.date()) else 0)
  return df

database(df)

In [ ]:
df.head()

In [ ]:
inicio_2018 = dt.date(2018,1,1)
fin_2018 = dt.date(2018,7,1)
df_2018 = pd.DataFrame({'Fecha': pd.date_range(inicio_2018, fin_2018-dt.timedelta(days=1),freq='d')})
df_2018 = database(df_2018)
df_2018

In [ ]:
predict_2018 = df_2018.drop(['Fecha'], axis=1)
predict_2018

**División de base de datos entre entrenamiento y validación**

Ahora, se extraen los datos a usar para entrenamiento y para validation.

Training: desde 01/01/2012 hasta 31/12/2016.
Validation: desde 01/01/2017 hasta 31/12/2017.
Luego se separan las unidades para tener los valores objetivo con los que compararemos lso modelos.

In [ ]:
df_training = df[df['Anio'] <= 2016].drop(['Fecha'], axis=1)
training = df_training.drop(['Unidades'], axis=1)
training_target = df_training['Unidades']
print(training)
training_target

In [ ]:
df_validation = df[df['Anio'] == 2017].drop(['Fecha'], axis=1)
validation = df_validation.drop(['Unidades'], axis=1)
validation_target = df_validation['Unidades']
print(validation)
validation_target

In [ ]:
df_validation = df[df['Anio'] == 2017].drop(['Fecha'], axis=1)
validation = df_validation.drop(['Unidades'], axis=1)
validation_target = df_validation['Unidades']
print(validation)
validation_target

**Modelo de predicción de número de vehículos registrados utilizando Bosques Aleatorios**

El primer modelo a entrenar es el modelo de bosque aleatorio. Se entrena con los datos de entrenamiento y luego se hacen predicciones del training y validación para verificar la variación de R^2 entre entrenamiento y validación.

In [ ]:
#defining the RandomForestRegressor
RF_model=RandomForestRegressor()
 
RF_model.fit(training,training_target)
#testing
y_predicted_rf = RF_model.predict(training)
r2_training_rf = r2_score(training_target, y_predicted_rf)
r2_training_rf

In [ ]:
y_validation_rf = RF_model.predict(validation)
r2_validation_rf = r2_score(validation_target, y_validation_rf)
r2_validation_rf

In [ ]:
r2_training_rf - r2_validation_rf

Por último, se predicen las posibles unidades de vehículos registrados para el primer semestre del 2018.

In [ ]:
y_predicted_2018_rf = RF_model.predict(predict_2018)
df_2018['Prediccion_RF'] = y_predicted_2018_rf
df_2018

**Modelo de predicción de número de vehículos registrados utilizando Regresión Lineal**

Ahora, se entrena un modelo de regresión lineal y se valida para después realizar las predicciones.

In [ ]:
regr_model = linear_model.LinearRegression()
regr_model.fit(training, training_target)
y_predicted_r = regr_model.predict(training)
r2_training_r = r2_score(training_target, y_predicted_r)
r2_training_r

In [ ]:
y_validation_r = regr_model.predict(validation)
r2_validation_r = r2_score(validation_target, y_validation_r)
r2_validation_r

In [ ]:
r2_training_r - r2_validation_r

Por útlimo, se predicen las unidades de vehículos registrados para el 2018.

In [ ]:
y_predicted_2018_r = regr_model.predict(predict_2018)
df_2018['Prediccion_R'] = y_predicted_2018_r
df_2018

**Modelo de predicción de número de vehículos registrados utilizando Regresión Logística**

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(df2, target)

In [ ]:
y_predicted_lr = logreg.predict(df2)
y_predicted_lr

In [ ]:
r2_score(target, y_predicted_lr)

**Modelo de predicción de número de vehículos registrados utilizando Regresión Lasso**

Ahora, se trabaja con el modelo de regresión Lasso. Se entrena y se predicen los valores de Training y Validation.

In [ ]:
lasso = linear_model.Lasso()
lasso.fit(training, training_target)
y_predicted_lasso = lasso.predict(training)
r2_training_lasso = r2_score(training_target, y_predicted_lasso)
r2_training_lasso

In [ ]:
y_validation_lasso = lasso.predict(validation)
r2_validation_lasso = r2_score(validation_target, y_validation_lasso)
r2_validation_lasso

In [ ]:
r2_training_lasso - r2_validation_lasso

Por útlimo predecimos los valores para el 2018 con este modelo.

In [ ]:
y_predicted_2018_lasso = lasso.predict(predict_2018)
df_2018['Prediccion_Lasso'] = y_predicted_2018_lasso
df_2018

**Modelo de predicción de número de vehículos registrados utilizando Redes Neuronales con Regresión como salida**

In [ ]:
training.shape

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = training.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(training, training_target, epochs=45, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

In [ ]:
# Load wights file of the best model :
wights_file = 'Weights-042--218.54323.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
y_predicted_nn = NN_model.predict(training)
r2_training_nn = r2_score(training_target, y_predicted_nn)
r2_training_nn

In [ ]:
y_validation_nn = NN_model.predict(validation)
r2_validation_nn = r2_score(validation_target, y_validation_nn)
r2_validation_nn

In [ ]:
r2_training_nn-r2_validation_nn

In [ ]:
y_predicted_2018_nn = NN_model.predict(predict_2018)
df_2018['Prediccion_NN'] = y_predicted_2018_nn
df_2018

Otro

In [ ]:
df_2018[df_2018['Prediccion_R']<0]

In [ ]:
df_2018[df_2018['Prediccion_Lasso']<0]

In [ ]:
df2012= (df['Fecha'] >= '2012-01-01') & (df['Fecha'] <= '2012-12-31')
df2012=df.loc[df2012]
df2012

In [ ]:
df2013= (df['Fecha'] >= '2013-01-01') & (df['Fecha'] <= '2013-12-31')
df2013=df.loc[df2013]
df2013

In [ ]:
dftest= df.copy()

In [ ]:
dftest

In [ ]:
import pandas as pd
import seaborn as sns
from calendar import month_name as mn

# month list
months = mn[1:]

# convert the column to categorical and ordered
dftest['Month_Name'] = pd.Categorical(dftest['Month_Name'], categories=months, ordered=True)

# plot the data
p = sns.relplot(kind='line', data=dftest, x='Month_Name', y='Unidades', hue='Month_Name', aspect=2.5, marker='o')

In [ ]:
sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(14,8)})
ax = sns.lineplot(data=df2012, x ='Month_Name', y = 'Unidades', palette='viridis')
plt.show()

In [ ]:
sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(14,8)})

ax = sns.lineplot(data=df2013, x ='Month_Name', y = 'Unidades', palette='viridis',
                  legend='full', lw=3)

ax.xaxis.set_major_locator(ticker.MultipleLocator(4))
plt.legend(bbox_to_anchor=(1, 1))
plt.show()